The goal of this section is two-fold.

First, seeing which genomes contain the flanking regions/genes of our genes of interest——meaning, in which genomes are the flanking genes present at a reasonable distance.

Second, trying to detect non-coding homologs for our genes of interest by running a sensitive search of the region bounded by the flanking genes.

# **Identifying flanking genes and regions**

Our genes of interest aren't strictly genes, but representative members of gene families. Other members of their family might have different flanking regions.

To account for this possibility, we define "flanking genes" as the unique set of genes that surround the members of our gene family of interest.

To this end, we proceed in the following way:

1. First, identify all flanking genes, i.e. immediate upstream and downstream genes, for each member of the gene family in question
2. Replace the upstream and downstream gene IDs with their corresponding family/cluster ID
3. Extract the set of unique pairs of gene family IDs from this list

In addition to extracting flanking genes, we also extract sequences immediately upstream and downstream of our genes of interest. To this end, we choose (randomly) one of the genomes in which both flanks are present, and extract +/-500bp from gene edge. This is repeated for each pair of gene family IDs.

### **1. Setup**

In [ ]:
#For the next step, the mmseqs2 cluster names in the protein-cluster designation file needs to be replaced with the mmseqs-silix one

awk 'FNR==NR {rep[$2]=$1; next} {$1 = ($1 in rep ? rep[$1] : $1); print}' mmseqs_silix_reclustering/replacements.tsv Ecoli.all.filtered.prot.clusters.longestrepresentative.tsv > Ecoli.all.filtered.prot.clusters.longestrepresentative.replaced.tsv
cut -f1,2 -d " " ../Ecoli.all.filtered.prot.clusters.longestrepresentative.replaced.tsv | tail -n+2 | awk '{print $2"\t"$1}' > Ecoli.all.filtered.prot.clusters.longestrepresentative.replaced.formatted.tsv

#Furthermore, all proteins——including those three categories excluded from analyses at an earlier step——need to be clustered as well
#In my previous code I did the clustering with usearch, which is another similarity-based clustering tool
#This is fine for our purposes

usearch -sortbylength Ecoli.all.prot.pangenomedb.faa -fastaout Ecoli.all.prot.pangenomedb.sorted.faa -minseqlength 1
usearch -cluster_smallmem Ecoli.all.prot.pangenomedb.sorted.faa -id 0.8 -centroids Ecoli.all.prot.nr.faa -uc Ecoli.all.clusters.uc

#Convert the .uc to a a clustered tsv
cut -f9,10 Ecoli.all.clusters.uc | sort -u | awk -F '\t' '($2=="*")' | awk -F '\t' '{print $1"\t"$1}' > clusters.tsv
cut -f9,10 Ecoli.all.clusters.uc | sort -u | awk -F '\t' '($2!="*")' | awk -F '\t' '{print $1"\t"$2}' >> clusters.tsv

#The clusters.tsv file is a two-column gene-cluster file, similar to the one generated by mmseqs2


### Debugging

In [ ]:
#I don't know why I needed to make an all-protein database of the sort seen below
#This could either be required, or be a vestige of old code I had.
#I can't think of a reason why I would need to generate this alt protein file, but in case it's important: the code is retained below

cd ../Ecoli_gtfs
for i in $(ls *gtf | rev | cut -f3- -d "." | rev)
do
cat "$i".nr.gtf | gtf2bed | bedtools getfasta -s -name -fi /stor/work/Ochman/hassan/MS_Ecoli_ORFans_Ch3/Ecoli_genomes/"$i".fasta -bed - > "$i".CDS.faa
done

mkdir /stor/work/Ochman/hassan/MS_Ecoli_ORFans_Ch3/Ecoli_cds
mv *CDS.faa /stor/work/Ochman/hassan/MS_Ecoli_ORFans_Ch3/Ecoli_cds/

cd /stor/work/Ochman/hassan/MS_Ecoli_ORFans_Ch3/Ecoli_cds/
for i in $(ls *faa | rev | cut -f3- -d '.' | rev)
do
/stor/work/Ochman/hassan/tools/faTrans -stop "$i".CDS.faa "$i".prot.faa
done

mkdir /stor/work/Ochman/hassan/MS_Ecoli_ORFans_Ch3/Ecoli_prot
mv *prot.faa /stor/work/Ochman/hassan/MS_Ecoli_ORFans_Ch3/Ecoli_prot

#prep done

cd /stor/work/Ochman/hassan/MS_Ecoli_ORFans_Ch3/

#Just take the 450 Ecoli genomes with good lineages
for i in $(awk -F '\t' '($5<45)' /stor/work/Ochman/hassan/Ecoli_pangenome/500_ipp_lineagedesignations.tsv | cut -f1)
do
sed "s/>/>"$i"@/g" Ecoli_prot/"$i".prot.faa >> Ecoli.all.prot.faa
sed "s/>/>"$i"@/g" Ecoli_cds/"$i".CDS.faa >> Ecoli.all.CDS.faa
done

grep "^>" Ecoli.all.prot.nr.faa | grep --no-group-separator -F -A1 -f - Ecoli.all.CDS.faa > Ecoli.all.CDS.nr.faa

less Ecoli.all.prot.pangenomedb.faa

SyntaxError: invalid decimal literal (ipython-input-3165409109.py, line 24)

### **2. Making gene flanks tables for each gene of interest**

In [ ]:
#For subsequent steps, the gtfs need to be sorted

for i in $(ls ../Ecoli_gtfs/*nr.gtf)
do
bedtools sort -i $i > temp && mv temp $i
done

mkdir flank
cp Ecoli.all.filtered.prot.clusters.longestrepresentative.replaced.formatted.tsv flank
cd flank

#The next bit of script progresses in a few steps
#1. For each member in a gene family of interest, get its upstream and downstream gene
#2. Make a five-column tab-separated that lists the gene of interest, upstream gene, its cluster, downstream gene, its cluster
#3. Concatenate these rows, each corresponding to a member of the gene familiy, into a single file
#4. If any row contains fewer than five columns, that means that gene does not have both upstream and downstream flanking genes
#5. Retain each unique pair of family name for the upstream and downstream genes

#For each gene in the list of genes of interest (e.g., ORFan candidates)
#We loop over each of the members in their family
#Since the code is time-consuming, I put it in an "echo" wrapper below
#I'm showing an example for one gene of interest to demonstrate how the code looks
for i in $(grep "JSKL00000000@EMDEOKIC_02191(" Ecoli.all.filtered.prot.clusters.longestrepresentative.replaced.formatted.tsv | cut -f1 -d "(")
do
    genome_name=$(echo $i | cut -f1 -d "@"); ortholog_name=$(echo $i | cut -f2 -d "@"); #extract genome name from each gene ID
    gene_ortholog_gtf=$(grep -w $ortholog_name ../../Ecoli_gtfs/$genome_name.nr.gtf) #extract gtf line corresponding to that gene
    echo $i > JSKL00000000@EMDEOKIC_02191_temp #Start building five-column row. First column——gene name
    #Get the upstream gene
    echo $gene_ortholog_gtf | sed "s/ /%/9" | sed "s/ /%/9" | sed "s/ /\t/g" | sed "s/%/ /g" | bedtools closest -a - -b ../../Ecoli_gtfs/$genome_name.nr.gtf -D a -id -io | awk -F '\t' '($12=="CDS")' | cut -f 6 -d "\"" | sed "s/^/$genome_name@/g" >> "$i"_upstream
    #Add it to the growing table
    cat "$i"_upstream >> JSKL00000000@EMDEOKIC_02191_temp
    #Add its corresponding gene family/cluster designation as well
    grep -F -f "$i"_upstream ../../clusters.tsv | cut -f2 >> JSKL00000000@EMDEOKIC_02191_temp
    rm "$i"_upstream
    #Get the downstream gene
    echo $gene_ortholog_gtf | sed "s/ /%/9" | sed "s/ /%/9" | sed "s/ /\t/g" | sed "s/%/ /g" | bedtools closest -a - -b ../../Ecoli_gtfs/$genome_name.nr.gtf -D a -iu -io | awk -F '\t' '($12=="CDS")' | cut -f 6 -d "\"" | sed "s/^/$genome_name@/g" >> "$i"_downstream
    #Add it to the growing table
    cat "$i"_downstream >> JSKL00000000@EMDEOKIC_02191_temp
    #Add its corresponding gene family/cluster designation as well
    grep -F -f "$i"_downstream ../../clusters.tsv | cut -f2 >> JSKL00000000@EMDEOKIC_02191_temp
    rm "$i"_downstream
    if [ "$(wc -l < JSKL00000000@EMDEOKIC_02191_temp)" -eq 5 ]; then #Only consider cases in which the file has 5 rows
        paste -sd, JSKL00000000@EMDEOKIC_02191_temp | sed "s/,/\t/g" >> JSKL00000000@EMDEOKIC_02191_flank_analysis #Transform into five-column table
        grep -E '^([^@]*@){5}[^@]*$' JSKL00000000@EMDEOKIC_02191_flank_analysis | cut -f3,5 | sort -u > JSKL00000000@EMDEOKIC_02191_flanks #Retain the family ID of each unique pair of flanking genes
    fi
done

#Parallelized code:

for j in $(cut -f1 -d "(" ../step1_genusspecific_ORFan.replaced.txt)
do
    # Create the script file for each $j
    echo "#!/bin/bash" > "${j}.sh"  # Start each script with a shebang

    # Generate each line of the original script within an echo statement and append to the script file
    echo "for i in \$(grep \"$j(\" Ecoli.all.filtered.prot.clusters.longestrepresentative.replaced.formatted.tsv | cut -f1 -d \"(\")" >> "${j}.sh"  # Loop over clusters
    echo "do" >> "${j}.sh"
    echo "    genome_name=\$(echo \$i | cut -f1 -d \"@\"); ortholog_name=\$(echo \$i | cut -f2 -d \"@\");" >> "${j}.sh"
    echo "    gene_ortholog_gtf=\$(grep -w \$ortholog_name ../../Ecoli_gtfs/\$genome_name.nr.gtf)" >> "${j}.sh"
    echo "    echo \$i > ${j}_temp" >> "${j}.sh"
    echo "    echo \$gene_ortholog_gtf | sed \"s/ /%/9\" | sed \"s/ /%/9\" | sed \"s/ /\\t/g\" | sed \"s/%/ /g\" | bedtools closest -a - -b ../../Ecoli_gtfs/\$genome_name.nr.gtf -D a -id -io | awk -F '\\t' '(\$12==\"CDS\")' | cut -f 6 -d \"\\\"\" | sed \"s/^/\$genome_name@/g\" >> \"\$i\"_upstream" >> "${j}.sh"
    echo "    cat \"\$i\"_upstream >> ${j}_temp" >> "${j}.sh"
    echo "    grep -F -f \"\$i\"_upstream ../../clusters.tsv | cut -f2 >> ${j}_temp" >> "${j}.sh"
    echo "    rm \"\$i\"_upstream" >> "${j}.sh"
    echo "    echo \$gene_ortholog_gtf | sed \"s/ /%/9\" | sed \"s/ /%/9\" | sed \"s/ /\\t/g\" | sed \"s/%/ /g\" | bedtools closest -a - -b ../../Ecoli_gtfs/\$genome_name.nr.gtf -D a -iu -io | awk -F '\\t' '(\$12==\"CDS\")' | cut -f 6 -d \"\\\"\" | sed \"s/^/\$genome_name@/g\" >> \"\$i\"_downstream" >> "${j}.sh"
    echo "    cat \"\$i\"_downstream >> ${j}_temp" >> "${j}.sh"
    echo "    grep -F -f \"\$i\"_downstream ../../clusters.tsv | cut -f2 >> ${j}_temp" >> "${j}.sh"
    echo "    rm \"\$i\"_downstream" >> "${j}.sh"

    # Add the conditional part for checking the number of lines
    echo "    if [ \"\$(wc -l < ${j}_temp)\" -eq 5 ]; then" >> "${j}.sh"
    echo "        paste -sd, ${j}_temp | sed \"s/,/\\t/g\" >> ${j}_flank_analysis" >> "${j}.sh"
    echo "        grep -E '^([^@]*@){5}[^@]*\$' ${j}_flank_analysis | cut -f3,5 | sort -u > ${j}_flanks" >> "${j}.sh"
    echo "    fi" >> "${j}.sh"

    # Close the loop
    echo "done" >> "${j}.sh"
done

### **3. Extract flanking genes (geneflanks) and flanking regions (proxflanks)**

In [ ]:
#Make a helper file concatenating the sequences of all flanking genes so next steps go quicker
cat *_flanks | sed "s/\t/\n/g" | sort -u | grep -A1 --no-group-separator -F -f - ../../Ecoli.all.CDS.faa > all_flanks.CDS.faa

#Geneflanks:

#Extracting flanking genes is straightforward
#For each unique flanking gene pair, assign a line number
#Extract their CDSs
#The identifier should be of the form: GeneOfInterest_linenumber_up/down_FlankingGeneFamily

for j in $(ls *_flanks | rev | cut -f2- -d "_" | rev)
do
for i in $(cat -n "$j"_flanks | sed "s/^ *//g" | sed "s/\t/,/g")
do
linenumber=$(echo $i | cut -f1 -d ',')
echo $i | cut -f2 -d ',' | grep --no-group-separator -A1 -f - all_flanks.CDS.faa | sed "s/>/>"$j"_"$linenumber"_up_/g" >> geneflanks.faa
echo $i | cut -f3 -d ',' | grep --no-group-separator -A1 -f - all_flanks.CDS.faa | sed "s/>/>"$j"_"$linenumber"_down_/g" >> geneflanks.faa
done
done

#Proxflanks:

#To get sequences immediately upstream and downstream of the gene of interest:
#Identify a genome——any genome——just pick one at random——where the gene has both flanking genes
#Extract +/-500bp

for j in $(ls *_flanks | rev | cut -f2- -d "_" | rev)
do
#The next line takes one ortholog at random from each unique pair of flanking genes
for i in $(grep -E '^([^@]*@){5}[^@]*$' "$j"_flank_analysis | awk -F '\t' '{print $0,$3"_"$5}' | awk '!seen[$6]++' | cat -n | sed "s/^ *//g" | sed "s/\t/,/g" | cut -f1 -d " ")
do
linenumber=$(echo $i | cut -f1 -d ',')
ortholog_name=$(echo $i | cut -f2 -d ',' | cut -f2 -d "@" | cut -f1 -d "(")
genome_name=$(echo $i | cut -f2 -d ',' | cut -f1 -d "@")
grep -w $ortholog_name ../../Ecoli_gtfs/$genome_name.filtered.nr.gtf | sed "s/ \"/ \""$j"_"$linenumber"_/g" >> proxflanks_interim.gtf #Get a concatenated gtf
done
done

#Get the sequences
#The next line is required to extract the gene sequences from the gtf made above
#Since the gtf has genes from a bunch of genomes (indicated by its harboring contig in column 1), the multifasta identifier has to have contigs
seqkit fx2tab ../../Ecoli.all.genomes.faa | sed "s/\t$//g" | cut -f2- -d "@" | sed "s/^/>/g" | sed "s/\t/\n/g" > ../../Ecoli.all.genomes.temp.faa
#The $4>0 is part is to get rid of those cases in which the start codon dips to negative
#I'm making the choice to skip those that don't have 500bp on either edge
cat proxflanks_interim.gtf | awk -F '\t' '{OFS=FS}{print $1,$2,$3,$4-500,$4,$6,$7,$8,$9}' |  awk -F '\t' '($4>0)' | sed "s/ \"/ \"left500_/g" | gtf2bed | bedtools getfasta -s -name -fi ../../Ecoli.all.genomes.temp.faa -bed - > proxflanks.faa
cat proxflanks_interim.gtf | awk -F '\t' '{OFS=FS}{print $1,$2,$3,$5,$5+500,$6,$7,$8,$9}' | sed "s/ \"/ \"right500_/g" | gtf2bed | bedtools getfasta -s -name -fi ../../Ecoli.all.genomes.temp.faa -bed - >> proxflanks.faa

#Remove those that lack either or both flanks:
#The code below counts the number of commas per line after massaging it, and if there are no commas——that means at least one flank is missing
grep "^>" proxflanks.faa | sed "s/_/\t/" | awk -F'\t' '{ values[$2] = (values[$2] == "" ? $1 : values[$2] ", " $1) } END { for (value in values) { print value "\t" values[value] } }' | grep "," | cut -f 1 -d "(" | grep --no-group-separator -A1 -f - proxflanks.faa > interim
mv interim proxflanks.faa

SyntaxError: invalid decimal literal (ipython-input-3587605900.py, line 102)

### **Identifying genomes which contain both flanks at a reasonable distance**

Once we have the flank sequences for each of our genes, we search them (blastn) against our genome databases to identify those genomes in which both of them are present within a reasonable distance (no closer than half the length of the gene sequence, and no further than 10000bp). After attaching taxonomy information to these files, we'll know which genomes have the flanking regions of these genes. (I only measure this distance between the best hit for each flank sequence per genome)

This is how the code below is written, but I want to generate an additional set of files that sets the length at 5X the gene length, and gets rid of the minimal limit.

In [ ]:
#First, the blast searches (Time consuming)
blastn -query geneflanks.faa -db /stor/scratch/Ochman/hassan/100724_Complete_Genomes/Escherichia_db/Escherichia_excluded.genomes -outfmt '6 qseqid sseqid pident nident qcovhsp length mismatch gapopen gaps sstrand qstart qend sstart send qlen slen evalue bitscore' -num_threads 104 -max_target_seqs 109734 -max_hsps 1 -out geneflanks_extragenus_blastn
#109734
blastn -query geneflanks.faa -db /stor/scratch/Ochman/hassan/100724_Complete_Genomes/Escherichia_db/Ecoli_excluded.genomes -outfmt '6 qseqid sseqid pident nident qcovhsp length mismatch gapopen gaps sstrand qstart qend sstart send qlen slen evalue bitscore' -num_threads 104 -max_target_seqs 195119 -max_hsps 1 -out geneflanks_intragenus_blastn
#195119
blastn -query geneflanks.faa -db ../../Ecoli.all.genomes -outfmt '6 qseqid sseqid pident nident qcovhsp length mismatch gapopen gaps sstrand qstart qend sstart send qlen slen evalue bitscore' -num_threads 104 -max_target_seqs 86906 -max_hsps 1 -out geneflanks_pangenome_blastn
#86906
blastn -query proxflanks.faa -db /stor/scratch/Ochman/hassan/100724_Complete_Genomes/Escherichia_db/Escherichia_excluded.genomes -outfmt '6 qseqid sseqid pident nident qcovhsp length mismatch gapopen gaps sstrand qstart qend sstart send qlen slen evalue bitscore' -num_threads 104 -max_target_seqs 109734 -max_hsps 1 -out proxflanks_extragenus_blastn
blastn -query proxflanks.faa -db /stor/scratch/Ochman/hassan/100724_Complete_Genomes/Escherichia_db/Ecoli_excluded.genomes -outfmt '6 qseqid sseqid pident nident qcovhsp length mismatch gapopen gaps sstrand qstart qend sstart send qlen slen evalue bitscore' -num_threads 104 -max_target_seqs 195119 -max_hsps 1 -out proxflanks_intragenus_blastn
blastn -query proxflanks.faa -db ../../Ecoli.all.genomes -outfmt '6 qseqid sseqid pident nident qcovhsp length mismatch gapopen gaps sstrand qstart qend sstart send qlen slen evalue bitscore' -num_threads 104 -max_target_seqs 86906 -max_hsps 1 -out proxflanks_pangenome_blastn
#choice: max-hsps 1

#Make a list of genomes in which both flanks are found
#No pid, qidentity or evalue cutoffs applied
cat proxflanks_extragenus_blastn proxflanks_intragenus_blastn proxflanks_pangenome_blastn | cut -f-2 | rev | awk -F'\t' 'BEGIN{OFS=FS} {split($2,a,"_"); $2=a[1]"_"a[2]"\t"a[3]; for(i=4;i<=length(a);i++) $2=$2"_"a[i]; print}' | rev | sed "s/_/\t/1" | awk -F '\t' '{OFS=FS}{print $1,$2"%"$4}' | awk -F'\t' '{ values[$2] = (values[$2] == "" ? $1 : values[$2] ", " $1) } END { for (value in values) { print value "\t" values[value] } }' | egrep "left.*right|right.*left" | cut -f1 | awk -F'%' '{ values[$1] = (values[$1] == "" ? $2 : values[$1] ", " $2) } END { for (value in values) { print value "\t" values[value] } }' > proxflanks_targets.txt
cat geneflanks_extragenus_blastn geneflanks_intragenus_blastn geneflanks_pangenome_blastn | sed "s/_up_/_up%/g" | sed "s/_down_/_down%/g" | sed "s/\t/%/" | cut -f1 | cut -f1,3 -d "%" | sed "s/%/\t/g" | sed "s/_up/\tup/g" | sed "s/_down/\tdown/g" | awk -F '\t' '{print $2"\t"$1">"$3}' | sort -u | awk -F'\t' '{ values[$2] = (values[$2] == "" ? $1 : values[$2] ", " $1) } END { for (value in values) { print value "\t" values[value] } }' | egrep "up.*down|down.*up" | cut -f1 | rev | sed "s/>/%/1" | rev | awk -F'%' '{ values[$1] = (values[$1] == "" ? $2 : values[$1] ", " $2) } END { for (value in values) { print value "\t" values[value] } }' > geneflanks_targets.txt

#We know make files for each gene that lists each target genome in the form of a five-column table
#contig name, start position, end position, length, strand
for i in $(cut -f 1 proxflanks_targets.txt)
do
  echo "grep -P \"$i\t\" proxflanks_targets.txt | sed \"s/,/\\n/g\" | sed \"s/\t/\\n/g\" | sed \"s/^ *//g\" | tail -n+2 > ${i}_proxflanks_targetlist.txt"
  echo "cat proxflanks_extragenus_blastn proxflanks_intragenus_blastn proxflanks_pangenome_blastn | grep -w -F -f ${i}_proxflanks_targetlist.txt - | grep \"${i}_\" | cut -f2- -d \"_\" | sed 's/_/\t/3' | cut -f1,3- | awk -F '\t' '{OFS=\"\"}{print \$13,\"%\",\$14,\"%\",\$10,\"\t\",\$2}' | awk -F'\t' '{ values[\$2] = (values[\$2] == \"\" ? \$1 : values[\$2] \", \" \$1) } END { for (value in values) { print value \"\t\" values[value] } }' | sed \"s/%plus, /%/g\" | sed \"s/%minus, /%/g\" | sed \"s/%plus/\tplus/g\" | sed \"s/%minus/\tminus/g\" | sed \"s/%/,/g\" | sed \"s/\t/,/g\" | sed \"s/ //g\" | awk -F',' '{identifier = \$1; values = \$2 \",\" \$3 \",\" \$4 \",\" \$5; split(values, array, \",\"); asort(array); middle1 = array[2]; middle2 = array[3]; difference = middle2 - middle1; if (difference >= 0) { print identifier, middle1, middle2, difference, \$6; } else { print identifier, middle2, middle1, -difference, \$6; } }' > ${i}_proxflanks_intervalinfo"
done | split -l30

cat gene*blastn | sort -u > geneflanks_allblastn #for later

#Do the same with geneflanks:

for i in $(cut -f 1 geneflanks_targets.txt)
do
  echo "grep -P \"$i\t\" geneflanks_targets.txt | sed \"s/,/\\n/g\" | sed \"s/\t/\\n/g\" | sed \"s/^ *//g\" | tail -n+2 > ${i}_geneflanks_targetlist.txt"
  echo "cat geneflanks_extragenus_blastn geneflanks_intragenus_blastn geneflanks_pangenome_blastn | grep -w -F -f ${i}_geneflanks_targetlist.txt - | grep \"${i}_\" | sed \"s/_up_/\\t/g\" | sed \"s/_down_/\\t/g\" | cut -f1,3- | awk -F '\t' '{OFS=\"\"}{print \$13,\"%\",\$14,\"%\",\$10,\"\t\",\$2}' | awk -F'\t' '{ values[\$2] = (values[\$2] == \"\" ? \$1 : values[\$2] \", \" \$1) } END { for (value in values) { print value \"\t\" values[value] } }' | sed \"s/%plus, /%/g\" | sed \"s/%minus, /%/g\" | sed \"s/%plus/\tplus/g\" | sed \"s/%minus/\tminus/g\" | sed \"s/%/,/g\" | sed \"s/\t/,/g\" | sed \"s/ //g\" | awk -F',' '{identifier = \$1; values = \$2 \",\" \$3 \",\" \$4 \",\" \$5; split(values, array, \",\"); asort(array); middle1 = array[2]; middle2 = array[3]; difference = middle2 - middle1; if (difference >= 0) { print identifier, middle1, middle2, difference, \$6; } else { print identifier, middle2, middle1, -difference, \$6; } }' > ${i}_geneflanks_intervalinfo"
done | split -l30 - y

#Now compile prox and gene flank information together in the same intervalinfo file:

for i in $(ls *intervalinfo | rev | cut -f4- -d "_" | rev | sort -u)
do
ls "$i"_*intervalinfo | sed "s/^/cat /g" | bash >> "$i"_compiled_intervalinfo.txt
done

#Attach taxonomy info to each file
#Make an interim file that contains the taxonomy information for all target contigs
#Otherwise searching the massive database would take a long time
cut -f1 -d " " *_compiled_intervalinfo.txt | sort -u > alltaxa.compiled_intervalinfo.txt
cat /stor/scratch/Ochman/hassan/100724_Complete_Genomes/Ecoli_extragenus_genome_contig_taxa.reduced.noescherichia.tsv /stor/scratch/Ochman/hassan/100724_Complete_Genomes/Ecoli_intragenus_genome_contig_taxa.tsv /stor/scratch/Ochman/hassan/100724_Complete_Genomes/Ecoli_pangenome_genome_contig_taxa.tsv | grep -w -F -f alltaxa.compiled_intervalinfo.txt - | cut -f2- > alltaxa.compiled_intervalinfo.interim
sort -k1 alltaxa.compiled_intervalinfo.interim -o alltaxa.compiled_intervalinfo.interim

#Attach taxonomy information from the nascent file to the interval info files
for i in $(ls *_compiled_intervalinfo.txt | rev | cut -f3- -d "_" | rev)
do
sort -k1 "$i"_compiled_intervalinfo.txt -o "$i"_compiled_intervalinfo.txt
cut -f1 -d " " "$i"_compiled_intervalinfo.txt | sort -u > temp
grep -w -F -f temp alltaxa.compiled_intervalinfo.interim | sort -k1 | join -1 1 -2 1 - "$i"_compiled_intervalinfo.txt > "$i"_compiled_intervalinfo.taxa.txt
done

#Use this code to remove unwanted interval infos——the genomes in which the gene is already present based on prior blasts
#The code is also used to remove sequences from regular blastn stuff
#My original code detected flanks in a conservative way——because the goal was to just detect non-coding orthologs
#For the more liberal definition of flanks, see next "debugging" chunk

for i in $(ls *_compiled_intervalinfo.taxa.txt | rev | cut -f3- -d "_" | rev)
do
ortholog_name=$(echo $i | cut -f2 -d "@")
genome_name=$(echo $i | cut -f1 -d "@")
value=$(grep -w $ortholog_name ../../Ecoli_gtfs/$genome_name.filtered.nr.gtf | awk -F '\t' '{print $5-$4+1}')
sed "s/$/ $value/" "$i"_compiled_intervalinfo.taxa.txt > "$i"_compiled_intervalinfo.taxa.txt.2 #Append length
echo "$i" | sed "s/$/(/g" | grep -F -f - ../all_genes_of_interest.presence.tsv | cut -f2 -d " " |
grep -v -w -F -f - "$i"_compiled_intervalinfo.taxa.txt.2 |
awk '(($5>($7*0.5))&&($5<10000))' > "$i"_compiled_intervalinfo.taxa.final.txt #note the filters
done

wc -l *final.txt | grep " 0 " | rev | cut -f1 -d " " | rev | sed "s/^/rm /g" | bash


### Debugging

In [ ]:
#Let's make another set of files where flanks are defined more liberally

mkdir flank_alternative_111025
cd flank_alternative_111025
cp ../*_compiled_intervalinfo.taxa.txt .

for i in $(ls *_compiled_intervalinfo.taxa.txt | rev | cut -f3- -d "_" | rev)
do
ortholog_name=$(echo $i | cut -f2 -d "@")
genome_name=$(echo $i | cut -f1 -d "@")
value=$(grep -w $ortholog_name ../../../Ecoli_gtfs/$genome_name.filtered.nr.gtf | awk -F '\t' '{print $5-$4+1}')
sed "s/$/ $value/" "$i"_compiled_intervalinfo.taxa.txt > "$i"_compiled_intervalinfo.taxa.txt.2
echo "$i" | sed "s/$/(/g" | grep -F -f - ../../all_genes_of_interest.presence.tsv | cut -f2 -d " " | grep -v -w -F -f - "$i"_compiled_intervalinfo.taxa.txt.2 | awk '($5<($7*5))' > "$i"_compiled_intervalinfo.taxa.final.txt
done

wc -l *final.txt | grep " 0 " | rev | cut -f1 -d " " | rev | sed "s/^/rm /g" | bash

## **Identifying homologs within flanking regions**

In [ ]:
#To detect homologs in flank-bounded regions, we extract +/-300bp from both edges of the interval
for i in $(ls *_compiled_intervalinfo.taxa.final.txt | rev | cut -f3- -d "_" | rev)
do
awk '{print $1,$2,$3-300,$4+300,$5,$6,$7}' "$i"_compiled_intervalinfo.taxa.final.txt | awk '{$3=($3<1?1:$3); $4=($4<1?1:$4); print}' > "$i"_compiled_intervalinfo.taxa.extended.txt
done

#We then use samtools to extract the sequences from the respective genomes
#This is a time-consuming step, so it's placed within wrappers to parallelize
#See Preparing_database.sh to see how the individual_genomes directory was made

for variable in $(ls *_compiled_intervalinfo.taxa.extended.txt | rev | cut -f3- -d "_" | rev)
do
    command="grep \"plus\" ${variable}_compiled_intervalinfo.taxa.extended.txt | awk '{OFS=\"\"}{print \"samtools faidx /stor/scratch/Ochman/hassan/100724_Complete_Genomes/individual_genomes/\",\$1,\" \",\$1,\":\",\$3,\"-\",\$4}' | sed \"s/\$/ >> ${variable}_interval.faa/g\" && grep \"minus\" ${variable}_compiled_intervalinfo.taxa.extended.txt | awk '{OFS=\"\"}{print \"samtools faidx -i /stor/scratch/Ochman/hassan/100724_Complete_Genomes/individual_genomes/\",\$1,\" \",\$1,\":\",\$3,\"-\",\$4}' | sed \"s/\$/ >> ${variable}_interval.faa/g\""
    output_file="${variable}_joint_samtools.sh"
    bash -c "$command" > "$output_file"
done

ls *samtools.sh | sed "s/^/bash /g" > running.sh

/stor/work/Ochman/hassan/tools/parallelize_run.samtools.sh

###Sanity checks:
#check for empty sequences:
for i in *_interval.faa; do seqkit fx2tab $i | awk '(length($2)==0)'; done
#none, good

#check if interval sequences couldn't be extracted
for i in $(ls *_interval.faa | rev | cut -f2- -d "_" | rev); do grep "^>" "$i"_interval.faa | wc -l; wc -l "$i"_joint_samtools.sh; done | paste - - | awk '($1!=$2)'
#All were extractable

#Now turn the extracted sequences into blast databases:
for i in $(ls *_interval.faa | rev | cut -f2- -d "_" | rev)
do
makeblastdb -in "$i"_interval.faa -dbtype nucl -out "$i"_interval
done

#We then blastn the gene sequences against the extracted sequences:
#Since the databases are small(er)——only sequences bounded by flanks——we can get away with using a very sensitive search parameter, -word 7
for i in $(ls *_interval.faa | rev | cut -f2- -d "_" | rev)
do
ls "$i"_interval.faa | rev | cut -f2- -d "_" | rev | sed "s/$/(/g" | grep --no-group-separator -A1 -f - ../all_genes_of_interest.CDS.faa |
blastn -query - -db "$i"_interval -outfmt 0 -num_threads 72 -num_descriptions 13638 -num_alignments 13638 -evalue 200000 -out "$i"_interval_blastn -word_size 7
done

#Now we convert the blast output to a parseable mview file
export PERL5LIB=/stor/scratch/Ochman/hassan/genomics_toolbox/mview-1.67/lib/

#Since mview takes a while, we put the code in "echo" to parallelize it
for i in $(ls *_interval_blastn | rev | cut -f3- -d "_" | rev)
do
echo "/stor/scratch/Ochman/hassan/genomics_toolbox/mview-1.67/bin/mview -in blast ${i}_interval_blastn > ${i}_blastn_mviewed"
done > running.sh

/stor/work/Ochman/hassan/tools/parallelize_run.sh #helper script used for parallelization

#We now parse the mviewed file according to some criteria
#We get rid of any case where the total combined length of all alignments doesn't cover 50% of query length
#And also targets whose alignments don't cover the query by at least 50%
#Since there's no e-value cutoff, it's necessary to utilize a query coverage cutoff to get rid of spurious hits
#This might miss out on very small regions of homologous non-coding sequences
#But that's fine, since we're independently collecting data on which of these genes had flanks at reasonable distances
for i in $(ls *mviewed | rev | cut -f3- -d "_" | rev)
do
querylength=$(echo $i | sed "s/$/(/g" | grep -F -A1 -f - ../all_genes_of_interest.CDS.faa | grep -v "^>" | awk '{print length($0)}')
ratio=$(tail -n+8 "$i"_blastn_mviewed | head -1 | awk '{print $(NF-1)}' | sed "s/:/\t/g" | awk -v var=$querylength -F '\t' '{print ($2-$1+1)/var}')
if (( $(echo "$ratio > 0.5" | bc -l) ))
then
tail -n+9 "$i"_blastn_mviewed | head -n-3 | awk '{print $2,$(NF-4),$NF}' | sed "s/%//g" | awk '($2>50)' | awk '{print $1,$3}' | sed "s/^/>/g" | sed "s/ /\n/g" > "$i"_blastn_seq.faa
fi
done

wc -l *_blastn_seq.faa | grep " 0 " | rev | cut -f1 -d " " | rev | sed "s/^/rm /g" | bash

#Adding taxonomic info back exactly as I did above:

#Attach taxonomy info to each file
#make an interim file containing contig names from all these files
cat *_blastn_seq.faa | grep "^>" | tr -d ">" | rev | cut -f2- -d ":" | rev | sort -u > alltaxa.compiled_intervalinfo.txt
cat /stor/scratch/Ochman/hassan/100724_Complete_Genomes/Ecoli_extragenus_genome_contig_taxa.reduced.noescherichia.tsv /stor/scratch/Ochman/hassan/100724_Complete_Genomes/Ecoli_intragenus_genome_contig_taxa.tsv /stor/scratch/Ochman/hassan/100724_Complete_Genomes/Ecoli_pangenome_genome_contig_taxa.tsv | grep -w -F -f alltaxa.compiled_intervalinfo.txt - | cut -f2- > alltaxa.compiled_intervalinfo.interim
sort -k1 alltaxa.compiled_intervalinfo.interim -o alltaxa.compiled_intervalinfo.interim

#Extract taxonomy info with this code
for i in $(ls *_blastn_seq.faa | rev | cut -f3- -d "_" | rev)
do
echo "seqkit fx2tab ${i}_blastn_seq.faa | sed \"s/\t$//g\" | awk -F \":\" '{print \$1\"\t\"\$0}' | sort -k1 > ${i}_blastn_seq.tsv"
echo "cut -f1 ${i}_blastn_seq.tsv | sort -u > ${i}.temp"
echo "grep -w -F -f ${i}.temp alltaxa.compiled_intervalinfo.interim | sort -k1 | join -1 1 -2 1 - ${i}_blastn_seq.tsv > ${i}_blastn_seq.interim.tsv"
done

for i in $(ls *_blastn_seq.interim.tsv | rev | cut -f3- -d "_" | rev)
do
awk '{print ">flank_"$2"_"$3,$4}' "$i"_blastn_seq.interim.tsv | sed "s/ /\n/g" > "$i"_blastn.seq.flank.faa #Mark the sequences with "flank"
done